# Softmax Regression

해당 repository 및 code들은 wikidocs에서 제공하는 'Pytorch로 시작하는 딥 러닝 입문'을 참고하였음을 밝힙니다. 해당 자료를 바탕으로 숙지한 개념과 추가적인 저의 생각을 기록할 계획입니다. 출처는 다음 사이트와 같습니다.<br><br>
* https://wikidocs.net/book/2788

# 01. 원-핫 인코딩(One-Hot Encoding)

범주형 데이터를 처리할 때 레이블을 표현하는 방법

아래는 세 개의 카테고리에 대해서 원-핫 인코딩을 통해서 레이블을 인코딩 했을 때 각 클래스 간의 제곱 오차가 균등함을 보여줍니다.<br>
<br>
((1,0,0) - (0,1,0))\*\*2 = (1-0)\*\*2 + (0-1)\*\*2 + (0-0)\*\*2 = 2<br>
((1,0,0) - (0,0,1))\*\*2 = (1-0)\*\*2 + (0-0)\*\*2 + (0-1)\*\*2 = 2<br>

다르게 표현하면 모든 클래스에 대해서 원-핫 인코딩을 통해 얻은 원-핫 벡터들은 모든 쌍에 대하여 유클리드 거리를 구해도 전부 유클리드 거리가 동일합니다. 원-핫 벡터는 이처럼 각 클래스의 표현 방법이 무작위성을 가진다는 점을 표현할 수 있습니다. 뒤에서 다시 언급되겠지만 이러한 원-핫 벡터의 관계의 무작위성은 때로는 단어의 유사성을 구할 수 없다는 단점으로 언급되기도 합니다.

# 02. 소프트맥스 회귀(Softmax Regression) 이해하기

* 이진 분류(Binary Classification): 로지스틱 회귀, 2개의 선택지 중에서 1개를 고르는 문제
* 다중 클래스 분류(Multi-Class Classification): 소프트맥스 회귀, 3개 이상의 선택지 중에서 1개를 고르는 문제

<img src='https://i.imgur.com/QaxsIhq.png' width='100%'>

### Cost Function: Cross Entropy Function

<img src='https://i.imgur.com/s9Ls2RK.png' width='100%'>

# 03. 소프트맥스 회귀의 비용 함수 구현하기

In [ ]:
import torch
import torch.nn.functional as F

In [ ]:
torch.manual_seed(1)

### 파이토치로 소프트맥스의 비용 함수 구현하기(로우-레벨)

In [ ]:
z = torch.FloatTensor([1,2,3])

In [ ]:
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [ ]:
hypothesis.sum()

tensor(1.)

In [ ]:
z = torch.rand(3, 5, requires_grad=True)

In [ ]:
# 각 샘플에 대해서 소프트맥스 함수를 적용하여야 하므로
# 두 번째 차원에 대해서 소프트맥스 함수를 적용한다는 의미에서 dim=1을 써줍니다.

hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)


In [ ]:
# 각 샘플에 대한 임의의 레이블 생성

y = torch.randint(5, (3,)).long()
print(y)

tensor([0, 2, 1])


In [ ]:
# 각 레이블에 대해서 원-핫 인코딩 수행
# 모든 원소가 0의 값을 가진 3x5 텐서 생성

y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

y.unsqueeze(1)를 하면 (3,)의 크기를 가졌던 y 텐서는 (3x1) 텐서가 됩니다. 즉, 다시 말해서 y.unsqueeze(1)의 결과는 아래와 같습니다.

In [ ]:
print(y.unsqueeze(1))

tensor([[0],
        [2],
        [1]])


그리고 scatter의 첫 번째 인자로 dim=1에 대해서 수행하라고 알려주고, 세 번째 인자에 숫자 1을 넣어줌으로써 두 번째 인자인 y_unsqueeze(1)이 알려주는 위치에 숫자 1을 넣도록 합니다. 앞서 연산 뒤에 \_를 붙이면 In-place Operation(덮어쓰기 연산)임을 배운 바 있습니다. 이에 따라서 y\_one\_hot의 최종 결과는 결국 아래와 같습니다.

In [ ]:
print(y_one_hot)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])


In [ ]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


### 파이토치로 소프트맥스의 비용 함수 구현하기(하이-레벨)

F.softmax() + torch.log() = F.log_softmax()

In [ ]:
# Low level

torch.log(F.softmax(z, dim=1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward>)

In [ ]:
# High level

F.log_softmax(z, dim=1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward>)

F.log_softmax() + F.nll\_loss() = F.cross_entropy()

In [ ]:
# Low level
# 첫 번째 수식

(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [ ]:
# 두 번째 수식

(y_one_hot * -F.log_softmax(z, dim=1)).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [ ]:
# High level
# 세 번째 수식
# nll == Negative Log Likelihood

F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.4689, grad_fn=<NllLossBackward>)

In [ ]:
# 네 번째 수식

F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward>)

F.cross\_entropy는 비용 함수에 소프트맥스 함수까지 포함하고 있음을 기억하고 있어야 구현 시 혼동하지 않습니다.

# 04. 소프트맥스 회귀 구현하기

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
torch.manual_seed(1)

In [ ]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

### 소프트맥스 회귀 구현하기(로우-레벨)

In [ ]:
print(x_train.shape)
print(y_train.shape)

torch.Size([8, 4])
torch.Size([8])


In [ ]:
y_one_hot = torch.zeros(8, 3)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
print(y_one_hot.shape)

torch.Size([8, 3])


In [ ]:
# 모델 초기화

W = torch.zeros((4,3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)      # Question: (1,3)이 올바른 설정 방법 아닌지?

# Optimizer 설정

optimizer = optim.SGD([W, b], lr=0.1)

In [ ]:
nb_epochs = 1000
for epoch in range(nb_epochs+ 1):

    # 가설
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)

    # 비용 함수
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost : {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost : 1.098612
Epoch  100/1000 Cost : 0.761050
Epoch  200/1000 Cost : 0.689991
Epoch  300/1000 Cost : 0.643229
Epoch  400/1000 Cost : 0.604117
Epoch  500/1000 Cost : 0.568255
Epoch  600/1000 Cost : 0.533922
Epoch  700/1000 Cost : 0.500291
Epoch  800/1000 Cost : 0.466908
Epoch  900/1000 Cost : 0.433507
Epoch 1000/1000 Cost : 0.399962


### 소프트맥스 회귀 구현하기(하이-레벨)

In [ ]:
# 모델 초기화
W = torch.zeros((4,3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)

    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433506
Epoch 1000/1000 Cost: 0.399962


### 소프트맥스 회귀 nn.Module로 구현하기

In [ ]:
# 모델을 선언 및 초기화
# 4개의 특성을 가지고 3개의 클래스로 분류
# input_dim=4, output_dim=3

model = nn.Linear(4, 3)

Linear의 결과값을 가지고 cross_entropy function에 대입하기 때문에 위와 같이 모델을 설정한다.

In [ ]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.616785
Epoch  100/1000 Cost: 0.658891
Epoch  200/1000 Cost: 0.573443
Epoch  300/1000 Cost: 0.518151
Epoch  400/1000 Cost: 0.473265
Epoch  500/1000 Cost: 0.433516
Epoch  600/1000 Cost: 0.396563
Epoch  700/1000 Cost: 0.360914
Epoch  800/1000 Cost: 0.325392
Epoch  900/1000 Cost: 0.289178
Epoch 1000/1000 Cost: 0.254148


### 소프트맥스 회귀 클래스로 구현하기

In [ ]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3)   # output이 3!

    def forward(self, x):
        return self.linear(x)

In [ ]:
model = SoftmaxClassifierModel()

In [ ]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 2.637636
Epoch  100/1000 Cost: 0.647903
Epoch  200/1000 Cost: 0.564643
Epoch  300/1000 Cost: 0.511043
Epoch  400/1000 Cost: 0.467249
Epoch  500/1000 Cost: 0.428281
Epoch  600/1000 Cost: 0.391924
Epoch  700/1000 Cost: 0.356742
Epoch  800/1000 Cost: 0.321577
Epoch  900/1000 Cost: 0.285617
Epoch 1000/1000 Cost: 0.250818
